In [78]:
import pandas as pd
import numpy as np
import gensim.utils as utils
import re


from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from gensim.models import Word2Vec

from sklearn.metrics.pairwise import cosine_similarity


from normalization import normalize_corpus
from utils import build_feature_matrix
import numpy as np

In [3]:
columns = ['index','text', 'tag']
def add_row(text, classes, df):
    df.loc[len(df)] = [len(df), utils.to_utf8(text, errors='replace').decode("utf8"), classes]

In [4]:
import textract
from os import listdir
from os.path import isfile, join


data = pd.DataFrame( columns=columns)
data

dirpath = 'data/train/shortstory/'
shortstory = [add_row(textract.process(dirpath + f),'shortstory', data) for f in listdir(dirpath) if isfile(join(dirpath, f))]

dirpath = 'data/train/java/'
java = [add_row(textract.process(dirpath + f), 'technical', data) for f in listdir(dirpath) if isfile(join(dirpath, f))]

dirpath = 'data/train/python/'
python = [add_row(textract.process(dirpath + f),'technical', data) for f in listdir(dirpath) if isfile(join(dirpath, f))]

dirpath = 'data/train/medicine/'
medicine = [add_row(textract.process(dirpath + f),'medicine', data) for f in listdir(dirpath) if isfile(join(dirpath, f))]

dirpath = 'data/train/mobilescreen/'
mobilescreen = [add_row(textract.process(dirpath + f),'mobile', data) for f in listdir(dirpath) if isfile(join(dirpath, f))]

dirpath = 'data/train/mobilememorycard/'
mobilememorycard = [add_row(textract.process(dirpath + f),'mobile', data) for f in listdir(dirpath) if isfile(join(dirpath, f))]

dirpath = 'data/train/lcd/'
mobilememorycard = [add_row(textract.process(dirpath + f),'lcd', data) for f in listdir(dirpath) if isfile(join(dirpath, f))]

dirpath = 'data/train/hippa/'
mobilememorycard = [add_row(textract.process(dirpath + f),'hippa', data) for f in listdir(dirpath) if isfile(join(dirpath, f))]

dirpath = 'data/train/srs/'
mobilememorycard = [add_row(textract.process(dirpath + f),'srs', data) for f in listdir(dirpath) if isfile(join(dirpath, f))]


In [5]:
columns = ['index','text', 'tag']
test = pd.DataFrame( columns=columns)
dirpath = 'data/test/medicine/'
[add_row(textract.process(dirpath + f),'medicine', test) for f in listdir(dirpath) if isfile(join(dirpath, f))]
dirpath = 'data/test/mobilescreen/'
[add_row(textract.process(dirpath + f),'mobile', test) for f in listdir(dirpath) if isfile(join(dirpath, f))]
dirpath = 'data/test/mobilememorycard/'
[add_row(textract.process(dirpath + f),'mobile', test) for f in listdir(dirpath) if isfile(join(dirpath, f))]
dirpath = 'data/test/lcd/'
[add_row(textract.process(dirpath + f),'lcd', test) for f in listdir(dirpath) if isfile(join(dirpath, f))]
dirpath = 'data/test/java/'
[add_row(textract.process(dirpath + f),'technical', test) for f in listdir(dirpath) if isfile(join(dirpath, f))]
dirpath = 'data/test/hippa/'
[add_row(textract.process(dirpath + f),'hippa', test) for f in listdir(dirpath) if isfile(join(dirpath, f))]
dirpath = 'data/test/mobile/'
[add_row(textract.process(dirpath + f),'mobile', test) for f in listdir(dirpath) if isfile(join(dirpath, f))]
dirpath = 'data/test/srs/'
[add_row(textract.process(dirpath + f),'srs', test) for f in listdir(dirpath) if isfile(join(dirpath, f))]


[None, None, None]

In [18]:
def verify_tokenize_xml(text):
#    text = '<guestbook><guest><fname>Terje</fname><lname>Beck</lname></guest><guest><fname>Jan</fname><lname>Refsnes</lname></guest><guest><fname>Torleif</fname><lname>Rasmussen</lname></guest>'
    xml_pattern = '<([^<>]+)>([^<>]*)<(\/[^<>]+)>'
    if re.search(xml_pattern, text):
        text = re.sub('<',' <', text)
        text = re.sub('>','> ', text)
        print('in xml')
        return re.split(' |\n', text) 
    else :
        return []


In [19]:
def verify_tokenize_hippa(text):
#    text = 'ISA*00*          *00*          *ZZ*EMEDNYBAT      *ZZ*ETIN           *100101*1000*^*00501*006000600*0*T*:~GS*HP*EMEDNYBAT*ETIN*20100101*1050*6000600*X*005010X221A1~ST*835*1740~BPR*H*0*C*NON************20100101~TRN*1*10100000000*1000000000~REF*EV*ETIN~DTM*405*20100101~N1*PR*NYSDOH~N3*OFFICE OF HEALTH INSURANCE PROGRAMS*CORNING TOWER, EMPIRE STATE PLAZA~N4*ALBANY*NY*122370080~PER*BL*PROVIDER SERVICES*TE*8003439000*UR*www.emedny.org~N1*PE*MAJOR MEDICAL PROVIDER*XX*9999999995~REF*TJ*000000000~LX*1~CLP*PATIENT ACCOUNT NUMBER*2*34*0**MC*1000220000000030*11~NM1*QC*1*SUBMITTED LAST*SUBMITTED FIRST****MI*LL88888L~NM1*74*1*CORRECTED LAST*CORRECTED FIRST~REF*EA*PATIENT ACCOUNT NUMBER~DTM*232*20100101~DTM*233*20100101~SVC*HC:V2020*12*0**0~DTM*472*20100101~CAS*CO*29*12~SVC*HC:V2103*22*0**0~DTM*472*20100101~CAS*CO*29*22~SE*25*1740~GE*1*6000600~IEA*1*006000600~'
    hippa_pattern = '[[^a-zA-Z0-9]*/*[[^a-zA-Z0-9., -_]*/*[^a-zA-Z0-9., -_]*]*~]*'    
    if re.match(hippa_pattern, text)    :
        text = re.sub('\*','\* ', text)
        text = re.sub('~',' ~ ', text)
        print('in hippa')
        return re.split(' |\n', text) 
    else :
        return []
  

In [24]:
def verify_tokenize_log(text):
#    text = "172.16.0.3 - - [25/Sep/2002:14:04:19 +0200] \"GET / HTTP/1.1\" 401 - \"\" \"Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.1) Gecko/20020827"
#    text = "192.168.2.32 - - [10/Feb/2017:13:02:23 +0530] \"GET / HTTP/1.0\" 200 45"
    log_pattern1 = r"(.*): (.*): (.*)"
    log_pattern2 = r"((\[[^\]]+\]) (\[[^\]]+\]) (\[[^\]]+\]) (.*): (.*))"
#    log_pattern3 = r"\[(.*?)\]|(\S+)"
    log_pattern5 = r"(.*?)-\[(.*?)\](.*)"
#    log_pattern4 = r"^(\[[^\]]+\]) (\[[^\]]+\]) (\[[^\]]+\]) (.*)$"
    pattern = "|".join([log_pattern1, log_pattern2, log_pattern5])

    if re.search(pattern, text):
        print('in log')
        return re.split(pattern , text) 
    else :
        return []

#verify_tokenize_log('192.168.2.32 - - [10/Feb/2017:13:02:23 +0530] \"GET / HTTP/1.0\" 200 45')

In [25]:
def review_to_wordlist( text):
    words = verify_tokenize_xml(text)
    if len(words) == 0:
        words = verify_tokenize_hippa(text)
        if len(words) == 0:
            words = verify_tokenize_log(text)
            if len(words) == 0:
                review_text = re.sub("[^a-zA-Z0-9]"," ", text)
                words = review_text.lower().split()
                print('in words')
#                words = [w for w in words if not w in cashedstopwords]
    return([word for word in words if not word in cashedstopwords ])

In [32]:
import nltk.data

tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
import string

printable = set(string.printable)
cashedstopwords = stopwords.words("english")
cashedstopwords.extend(['', None])
def review_to_sentences( text):
    text = filter(lambda x: x in printable, text.strip())
    raw_sentences = tokenizer.tokenize(text)
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.extend( review_to_wordlist( raw_sentence ))
    return sentences

### Normalize train Data

In [80]:
norm_corpus = normalize_corpus(data.text, lemmatize=True)
tfidf_vectorizer, tfidf_features = build_feature_matrix(norm_corpus,
                                                        feature_type='tfidf',
                                                        ngram_range=(1, 1), 
                                                        min_df=0.0, max_df=1.0)

### Normalize test Data

In [81]:
# normalize and extract features from the query corpus
norm_query_docs =  normalize_corpus(test.text, lemmatize=True)            
query_docs_tfidf = tfidf_vectorizer.transform(norm_query_docs)


### Cosine Similarity

In [86]:
def compute_cosine_similarity(doc_features, corpus_features,
                              top_n=3):
    # get document vectors
    doc_features = doc_features.toarray()[0]
    corpus_features = corpus_features.toarray()
    # compute similarities
    similarity = np.dot(doc_features, 
                        corpus_features.T)
    # get docs with highest similarity scores
    top_docs = similarity.argsort()[::-1][:top_n]
    top_docs_with_score = [(index, round(similarity[index], 3))
                            for index in top_docs]
    return top_docs_with_score

    
print 'Document Similarity Analysis using Cosine Similarity'
print '='*60
for index, doc in enumerate(test.text):
    
    doc_tfidf = query_docs_tfidf[index]
    top_similar_docs = compute_cosine_similarity(doc_tfidf,
                                             tfidf_features,
                                             top_n=5)
    print 'Document',index+1 ,':' , test.tag[index]
    print 'Top', len(top_similar_docs), 'similar docs:'
    print '-'*40 
    for doc_index, sim_score in top_similar_docs:
        print 'Doc num: {} Similarity Score: {}  tag: {}'.format(doc_index+1,
                                                                 sim_score,
                                                       data.tag[doc_index])  
        print '-'*40       
    print                                                      
    

Document Similarity Analysis using Cosine Similarity
Document 1 : medicine
Top 5 similar docs:
----------------------------------------
Doc num: 40 Similarity Score: 0.312  tag: medicine
----------------------------------------
Doc num: 42 Similarity Score: 0.234  tag: medicine
----------------------------------------
Doc num: 38 Similarity Score: 0.222  tag: medicine
----------------------------------------
Doc num: 48 Similarity Score: 0.216  tag: medicine
----------------------------------------
Doc num: 52 Similarity Score: 0.209  tag: medicine
----------------------------------------

Document 2 : medicine
Top 5 similar docs:
----------------------------------------
Doc num: 49 Similarity Score: 0.318  tag: medicine
----------------------------------------
Doc num: 51 Similarity Score: 0.318  tag: medicine
----------------------------------------
Doc num: 53 Similarity Score: 0.182  tag: medicine
----------------------------------------
Doc num: 50 Similarity Score: 0.182  tag: me

### Hellinger Bhattacharya Distance

In [88]:
def compute_hellinger_bhattacharya_distance(doc_features, corpus_features,
                                            top_n=3):
    # get document vectors                                            
    doc_features = doc_features.toarray()[0]
    corpus_features = corpus_features.toarray()
    # compute hb distances
    distance = np.hstack(
                    np.sqrt(0.5 *
                            np.sum(
                                np.square(np.sqrt(doc_features) - 
                                          np.sqrt(corpus_features)), 
                                axis=1)))
    # get docs with lowest distance scores                            
    top_docs = distance.argsort()[:top_n]
    top_docs_with_score = [(index, round(distance[index], 3))
                            for index in top_docs]
    return top_docs_with_score 

print 'Document Similarity Analysis using Hellinger-Bhattacharya distance'
print '='*60
for index, doc in enumerate(test.text):
    
    doc_tfidf = query_docs_tfidf[index]
    top_similar_docs = compute_hellinger_bhattacharya_distance(doc_tfidf,
                                             tfidf_features,
                                             top_n=5)
    print 'Document',index+1 ,':',  test.tag[index]
    print 'Top', len(top_similar_docs), 'similar docs:'
    print '-'*40 
    for doc_index, sim_score in top_similar_docs:
        print 'Doc num: {} Similarity Score: {}  tag: {}'.format(doc_index+1,
                                                                 sim_score,
                                                       data.tag[doc_index]) 
        print '-'*40       
    print                 



Document Similarity Analysis using Hellinger-Bhattacharya distance
Document 1 : medicine
Top 5 similar docs:
----------------------------------------
Doc num: 84 Similarity Score: 2.533  tag: lcd
----------------------------------------
Doc num: 36 Similarity Score: 2.551  tag: medicine
----------------------------------------
Doc num: 40 Similarity Score: 2.563  tag: medicine
----------------------------------------
Doc num: 92 Similarity Score: 2.574  tag: lcd
----------------------------------------
Doc num: 65 Similarity Score: 2.606  tag: mobile
----------------------------------------

Document 2 : medicine
Top 5 similar docs:
----------------------------------------
Doc num: 36 Similarity Score: 2.488  tag: medicine
----------------------------------------
Doc num: 50 Similarity Score: 2.517  tag: medicine
----------------------------------------
Doc num: 53 Similarity Score: 2.517  tag: medicine
----------------------------------------
Doc num: 92 Similarity Score: 2.519  tag: 

### BM25 Similarity

In [91]:
import scipy.sparse as sp 

def compute_corpus_term_idfs(corpus_features, norm_corpus):
    
    dfs = np.diff(sp.csc_matrix(corpus_features, copy=True).indptr)
    dfs = 1 + dfs # to smoothen idf later
    total_docs = 1 + len(norm_corpus)
    idfs = 1.0 + np.log(float(total_docs) / dfs)
    return idfs

def compute_bm25_similarity(doc_features, corpus_features,
                            corpus_doc_lengths, avg_doc_length,
                            term_idfs, k1=1.5, b=0.75, top_n=3):
    # get corpus bag of words features
    corpus_features = corpus_features.toarray()
    # convert query document features to binary features
    # this is to keep a note of which terms exist per document
    doc_features = doc_features.toarray()[0]
    doc_features[doc_features >= 1] = 1
    
    # compute the document idf scores for present terms
    doc_idfs = doc_features * term_idfs
    # compute numerator expression in BM25 equation
    numerator_coeff = corpus_features * (k1 + 1)
    numerator = np.multiply(doc_idfs, numerator_coeff)
    # compute denominator expression in BM25 equation
    denominator_coeff =  k1 * (1 - b + 
                                (b * (corpus_doc_lengths / 
                                        avg_doc_length)))
    denominator_coeff = np.vstack(denominator_coeff)
    denominator = corpus_features + denominator_coeff
    # compute the BM25 score combining the above equations
    bm25_scores = np.sum(np.divide(numerator,
                                   denominator),
                         axis=1)
    # get top n relevant docs with highest BM25 score                     
    top_docs = bm25_scores.argsort()[::-1][:top_n]
    top_docs_with_score = [(index, round(bm25_scores[index], 3))
                            for index in top_docs]
    return top_docs_with_score

vectorizer, corpus_features = build_feature_matrix(norm_corpus,
                                                   feature_type='frequency')
query_docs_features = vectorizer.transform(norm_query_docs)

doc_lengths = [len(doc.split()) for doc in norm_corpus]   
avg_dl = np.average(doc_lengths) 
corpus_term_idfs = compute_corpus_term_idfs(corpus_features,
                                            norm_corpus)

print 'Document Similarity Analysis using BM25'
print '='*60
for index, doc in enumerate(test.text):
    
    doc_features = query_docs_features[index]
    top_similar_docs = compute_bm25_similarity(doc_features,
                                               corpus_features,
                                               doc_lengths,
                                               avg_dl,
                                               corpus_term_idfs,
                                               k1=1.5, b=0.75,
                                               top_n=2)
    print 'Document',index+1 ,':', test.tag[index]
    print 'Top', len(top_similar_docs), 'similar docs:'
    print '-'*40 
    for doc_index, sim_score in top_similar_docs:
        print 'Doc num: {} BM25 Score: {}\nDoc: {}'.format(doc_index+1,
                                                                 sim_score,
                                                                 data.tag[doc_index])  
        print '-'*40       
    print


Document Similarity Analysis using BM25
Document 1 : medicine
Top 2 similar docs:
----------------------------------------
Doc num: 20 BM25 Score: 1266.218
Doc: technical
----------------------------------------
Doc num: 12 BM25 Score: 1169.351
Doc: technical
----------------------------------------

Document 2 : medicine
Top 2 similar docs:
----------------------------------------
Doc num: 4 BM25 Score: 1426.937
Doc: shortstory
----------------------------------------
Doc num: 20 BM25 Score: 1387.352
Doc: technical
----------------------------------------

Document 3 : medicine
Top 2 similar docs:
----------------------------------------
Doc num: 4 BM25 Score: 1410.91
Doc: shortstory
----------------------------------------
Doc num: 20 BM25 Score: 1366.072
Doc: technical
----------------------------------------

Document 4 : medicine
Top 2 similar docs:
----------------------------------------
Doc num: 20 BM25 Score: 1066.886
Doc: technical
----------------------------------------
Doc